In this lab, we will try to use the OpenNMT library to train an NMT model using the toy English-German dataset.

This notebook was found originally at:
https://github.com/OpenNMT/OpenNMT-py#quickstart

In [ ]:
# Install OpenNMT-py 2.x
# NOTE: By the end of the insatallation, it might ask for restarting the runtime...
# In this case, just click the "RESTART RUNTIME" button.

!pip3 install git+https://github.com/OpenNMT/OpenNMT-py.git

  Running command git clone -q https://github.com/OpenNMT/OpenNMT-py.git 'C:\Users\vsvik\AppData\Local\Temp\pip-req-build-11uqk6l8'



  Cloning https://github.com/OpenNMT/OpenNMT-py.git to c:\users\vsvik\appdata\local\temp\pip-req-build-11uqk6l8
  Resolved https://github.com/OpenNMT/OpenNMT-py.git to commit fc3409ce08785915728dac9e9c9fb49f73027917
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached rsa-4.8-py3-none-any.whl (39 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Created wheel for OpenNMT-py: filename=OpenNMT_py-2.2.0-py3-none-any.whl size=222756 sha256=4eb5e93f4e18e3ee1c5abcf754c0e7ffd296cfd586a4ec54e16188c629846ce7
  Stored in directory: C:\Users\vsvik\AppData\Local\Temp\pip-ephem-wheel-cache-otd1pts_\wheels\ca\98\c3\5b4cbbdb1d595b4bf163488914782db68c3a7fac8e98857d3c
Successfully built OpenNMT-py


In [ ]:
# On Google Colab ONLY
# Reinstall Torch to avoid incompatibility with Cuda 10.1

# NOTE: By the end of the insatallation, it might ask for restarting the runtime...
# In this case, just click the "RESTART RUNTIME" button.

!pip3 install --ignore-installed torch==1.6.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 552.8 MB 4.5 kB/s 
     |████████████████████████████████| 829 kB 5.3 MB/s 
     |████████████████████████████████| 15.7 MB 37.3 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=edcdc2b6a6522061e82850c57cdb27004c29c129963d1c5f7505f1954b911de8
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.6.0+cu92 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.6.0+cu92 which is incompatible.
torchau

In [ ]:
# Download the files of the QuickStart

!wget https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
!tar xf toy-ende.tar.gz

--2022-05-13 08:17:46--  https://s3.amazonaws.com/opennmt-trainingdata/toy-ende.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.84.222
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.84.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1662081 (1.6M) [application/x-gzip]
Saving to: ‘toy-ende.tar.gz’

toy-ende.tar.gz     100%[===================>]   1.58M  8.37MB/s    in 0.2s    

2022-05-13 08:17:46 (8.37 MB/s) - ‘toy-ende.tar.gz’ saved [1662081/1662081]



In [ ]:
# Optional: List the extracted files

!cd toy-ende/ && ls

src-test.txt   src-val.txt   tgt-train.txt
src-train.txt  tgt-test.txt  tgt-val.txt


In [ ]:
# Optional: Print the first 3 lines of the source file

!head -n 3 toy-ende/src-train.txt

It is not acceptable that , with the help of the national bureaucracies , Parliament &apos;s legislative prerogative should be made null and void by means of implementing provisions whose content , purpose and extent are not laid down in advance .
Federal Master Trainer and Senior Instructor of the Italian Federation of Aerobic Fitness , Group Fitness , Postural Gym , Stretching and Pilates; from 2004 , he has been collaborating with Antiche Terme as personal Trainer and Instructor of Stretching , Pilates and Postural Gym .
&quot; Two soldiers came up to me and told me that if I refuse to sleep with them , they will kill me . They beat me and ripped my clothes .


In [ ]:
# Optional: Check the number of lines in the source file

!echo "Number of lines:" && wc -l toy-ende/src-train.txt

Number of lines:
10000 toy-ende/src-train.txt


In [ ]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano

config = '''# toy_en_de.yaml

## Where the samples will be written
save_data: toy-ende/run/example

## Where the vocab(s) will be written
src_vocab: toy-ende/run/example.vocab.src
tgt_vocab: toy-ende/run/example.vocab.tgt

## Where the model will be saved
save_model: model/model

# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: toy-ende/src-train.txt
        path_tgt: toy-ende/tgt-train.txt
    valid:
        path_src: toy-ende/src-val.txt
        path_tgt: toy-ende/tgt-val.txt

world_size: 1
gpu_ranks: [0]

# Remove or modify these lines for bigger files
train_steps: 1000
valid_steps: 200
'''

with open("toy_en_de.yaml", "w+") as config_yaml:
  config_yaml.write(config)

!cat toy_en_de.yaml

# toy_en_de.yaml

## Where the samples will be written
save_data: toy-ende/run/example

## Where the vocab(s) will be written
src_vocab: toy-ende/run/example.vocab.src
tgt_vocab: toy-ende/run/example.vocab.tgt

## Where the model will be saved
save_model: model/model

# Prevent overwriting existing files in the folder
overwrite: False

# Corpus opts:
data:
    corpus_1:
        path_src: toy-ende/src-train.txt
        path_tgt: toy-ende/tgt-train.txt
    valid:
        path_src: toy-ende/src-val.txt
        path_tgt: toy-ende/tgt-val.txt

world_size: 1
gpu_ranks: [0]

# Remove or modify these lines for bigger files
train_steps: 1000
valid_steps: 200


In [ ]:
!pip install --upgrade OpenNMT-py==2.0.0rc1

     |████████████████████████████████| 219 kB 5.4 MB/s 
     |████████████████████████████████| 53 kB 1.7 MB/s 
     |████████████████████████████████| 16.6 MB 28.9 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0


In [ ]:
# Build Vocabulary

!onmt_build_vocab -config toy_en_de.yaml -n_sample -1

Corpus corpus_1's weight should be given. We default it to 1 for you.
Traceback (most recent call last):
  File "/usr/local/bin/onmt_build_vocab", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", line 63, in main
    build_vocab_main(opts)
  File "/usr/local/lib/python3.7/dist-packages/onmt/bin/build_vocab.py", line 23, in build_vocab_main
    ArgumentParser.validate_prepare_opts(opts, build_vocab_only=True)
  File "/usr/local/lib/python3.7/dist-packages/onmt/utils/parse.py", line 130, in validate_prepare_opts
    cls._validate_vocab_opts(opt, build_vocab_only=build_vocab_only)
  File "/usr/local/lib/python3.7/dist-packages/onmt/utils/parse.py", line 98, in _validate_vocab_opts
    cls._validate_file(opt.src_vocab, info='src vocab')
  File "/usr/local/lib/python3.7/dist-packages/onmt/utils/parse.py", line 18, in _validate_file
    raise IOError(f"Please check path of your {info} file!")
OSError: Please check path of your 

In [ ]:
!pwd

/content


In [ ]:
# Check if GPU is active
# If not, go to "Runtime" menu > "Change runtime type" > "GPU"

!nvidia-smi -L

In [ ]:
# Make sure the GPU is visable to PyTorch

import torch

gpu_id = torch.cuda.current_device()
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(gpu_id))

In [ ]:
# Train the NMT model

!onmt_train -config toy_en_de.yaml

In [ ]:
# Translate

!onmt_translate -model model/model_step_1000.pt -src toy-ende/src-test.txt -output toy-ende/pred_1000.txt -gpu 0 -verbose

Install Sacrebleu to evaluate the model

In [ ]:
!pip install sacrebleu

In [ ]:
!sacrebleu toy-ende/tgt-test.txt < toy-ende/pred_1000.txt

## Assignment

 #### A1 
  - Please note down the BLEU scores obtained above in the cell below.


* (note down results here)

#### A2 

 - Your assginment is to train a model using the OpenNMT library as shown above but with larger dataset.

 - You can use any parallel corpus available from [Samanantar](https://indicnlp.ai4bharat.org/samanantar/)

 - Train a model on a single language pair and evaluate it using BLEU score as a metric as shown above.

 - Also note down the hyperparameters used for training the model. 

 - As a class you can discuss amongst yourselves and can collectively try different hyperparameters. 

 - If the parallel corpus is hard to fit in the GPU memory then you can use a smaller dataset, but if you are collectively trying different hyperparameters then all of you should experiment with the same dataset.

 - (Optional) You can further try to byte-pair encode the corpus and re-train the model. [The byte-pair encoding code is available in this notebook.](https://github.com/cfiltnlp/IITB-English-Hindi-PC/blob/main/IITB_En_Hi_Get_Data.ipynb) This notebook contains the code for byte-pair encoding the [IITB-English Hindi Parallel Corpus](https://huggingface.co/datasets/cfilt/iitb-english-hindi)